In [20]:
# Carregar pacotes necessários
library(MASS)

# Carregar os dados
data <- read.csv("./main_rent_merge_2018ith_idh_wo_duplicates_2024_05_05_out.csv", dec = ",", header = TRUE, sep = ";")

# Definir as variáveis dependente e independentes
y <- data$price_m2
X <- data[, c("area_m2", "suite", "bathrooms", "garage", "condo_real", "metro_dist_km", "delta_cbd_farialima", "idh", "gym", "field_quadra", "elevator", "furnished", "swimming_pool")]

# Função para calcular o R² ajustado
adjusted_r_squared <- function(model, n, k) {
  1 - (1 - summary(model)$r.squared) * ((n - 1) / (n - k - 1))
}

# Função para aplicar a transformação de Box-Cox
box_cox_transformation <- function(y, theta) {
  if (theta == 0) {
    return(log(y))
  } else {
    return((y^theta - 1) / theta)
  }
}

# Testar diferentes combinações de theta e lambda
theta_values <- c(-1, 0, 1)
lambda_values <- c(-1, 0, 1)
best_r_squared <- -Inf
best_theta <- NA
best_lambda <- NA

for (theta in theta_values) {
  y_transformed <- box_cox_transformation(y, theta)

  for (lambda in lambda_values) {
    X_transformed <- apply(X, 2, function(x) box_cox_transformation(x, lambda))

    model <- lm(y_transformed ~ X_transformed)
    n <- nrow(data)
    k <- length(coef(model)) - 1
    r_squared <- adjusted_r_squared(model, n, k)

    if (r_squared > best_r_squared) {
      best_r_squared <- r_squared
      best_theta <- theta
      best_lambda <- lambda
    }
  }
}

cat("Melhor R² ajustado:", best_r_squared, "\n")
cat("Melhor valor de theta:", best_theta, "\n")
cat("Melhor valor de lambda:", best_lambda, "\n")


ERROR: Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...): NA/NaN/Inf in 'x'
